<a href="https://colab.research.google.com/github/Abyukth-DA-code/SCT_ML_3/blob/main/SCT_ML_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import zipfile

zip_path = "sct3.zip"
extract_dir = "sct3_extracted"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

data_dir = extract_dir   # main folder after extraction
categories = ["cats", "dogs"]   # match folder names exactly

X = []
y = []

IMG_SIZE = 64

for category in categories:
    folder = os.path.join(data_dir, category)
    label = categories.index(category)   # cats=0, dogs=1
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        try:
            img_arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)   # grayscale
            resized_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))  # resize
            X.append(resized_arr.flatten())   # flatten 64x64 -> 4096 vector
            y.append(label)
        except Exception:
            pass  # skip broken images

X = np.array(X)
y = np.array(y)

print("Dataset loaded:", X.shape, y.shape)

# 2. Split into train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Train SVM classifier
svm = SVC(kernel="linear", C=1.0)  # linear kernel works well for high-dim data
svm.fit(X_train, y_train)

# 4. Evaluate
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=categories))

Dataset loaded: (410, 4096) (410,)
Accuracy: 0.4146341463414634
              precision    recall  f1-score   support

        cats       0.47      0.53      0.50        45
        dogs       0.32      0.27      0.29        37

    accuracy                           0.41        82
   macro avg       0.40      0.40      0.40        82
weighted avg       0.40      0.41      0.41        82



In [2]:
import os

print("Contents of sct3 folder:", os.listdir("/content/sct3_extracted"))

Contents of sct3 folder: ['dogs', 'cats']


In [5]:
def predict_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"❌ Could not load image at {image_path}. Check the path.")

    resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    flattened = resized.flatten().reshape(1, -1)
    prediction = svm.predict(flattened)[0]
    return categories[prediction]

# Example usage (make sure the file exists at this path)
print("Prediction for sample image:", predict_image("/content/sct3_extracted/dogs/125.jpg"))

Prediction for sample image: dogs
